In [528]:
%reload_ext autoreload
%autoreload 2
from text_processing import *
import pandas as pd 
import numpy as np

### Notebook gets word frequencies
- Across countries, across all years

In [380]:
headlines = pd.read_csv('../data/processed/headlines_cl_sent_sm_rapi.csv')
headlines['year'] = pd.to_datetime(headlines['time']).dt.year
headlines.drop(columns = ['Unnamed: 0'], inplace = True)

In [381]:
nr_headlines = headlines[['country', 'year']]
nr_headlines['nr_headlines'] = 1
nr_headlines = nr_headlines.groupby(['country', 'year']).sum().reset_index()

<ipython-input-381-f791f03d8227>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nr_headlines['nr_headlines'] = 1


In [383]:
nr_headlines[nr_headlines['country']=='South Africa']

,country,year,nr_headlines
17,South Africa,2005,1
18,South Africa,2006,1
19,South Africa,2009,1
20,South Africa,2010,64
21,South Africa,2011,239
22,South Africa,2012,355
23,South Africa,2013,402
24,South Africa,2014,400
25,South Africa,2015,580
26,South Africa,2016,1010


In [385]:
# # get the headlines and apply text processing function on it
# # Takes a long time to run
# df = headlines[['country', 'year', 'headline_no_site']]
# df = df.drop_duplicates()
# df['clean_headline'] = df['headline_no_site'].apply(text_preprocessing) 

# df.to_csv('../data/processed/word_freq_new.csv')

array(['body', 'boy', 'boyfriend', ..., 'zoe', 'zolani', 'zondo'],
      dtype=object)

In [397]:
#(df2[df2['country']=='South Africa']['word'].unique())

In [82]:
%%time
import ast

df1 = df.copy()

# exclude nouns from word list
# excluded_tags = {"NOUN", "VERB", "ADJ", "ADV", "ADP", "PROPN"}
included_tags = {"VERB", "ADJ", "ADV", "NOUN", "ADP"}

# df["clean_h1_verbs_adjs"] = df['clean_hl_words'].apply(lambda x: [str(idx) for idx in nlp(' '.join(word for word in ast.literal_eval(x))) if idx.pos_ in included_tags])
df1["clean_h1_verbs_adjs"] = df1['clean_headline'].apply(lambda x: [str(idx) for idx in nlp(' '.join(word for word in (x))) if idx.pos_ in included_tags])

Wall time: 1h 10min 56s


In [84]:
df1.to_csv('../data/processed/word_freq_parts_of_speech.csv')

In [529]:
df1 = pd.read_csv('../data/processed/word_freq_parts_of_speech.csv')
df1.drop(columns = 'Unnamed: 0', inplace = True)
df1['clean_h1_verbs_adjs'] = df1['clean_h1_verbs_adjs'].apply(lambda x: ast.literal_eval(x))

df1 = df1[['country', 'year', 'clean_h1_verbs_adjs']]

df1 = df1.explode('clean_h1_verbs_adjs')
df1 = df1[['country', 'year', 'clean_h1_verbs_adjs']]
df1['count'] = 1
#df1
df1.columns = ['country', 'year', 'word', 'count']
df1 = df1.groupby(['country', 'year', 'word']).sum().reset_index()

df1 = df1[~(df1.word.isin(['|', "'", '&', '', '!', '#', '%', '&', '+', ' ', '.']))]
df1 = remove_words(df1)

df1 = df1[df1['count']>1]

df1 = df1[(df1['word'].apply(remove_numbers))]
df1 = df1[(df1['word'].apply(remove_special_characters))]
df1 = df1[(df1['word'].apply(remove_two_letter_words))]


In [530]:
# Most used words across all years
word_freq_allyears = df1[['country', 'word', 'count']]
word_freq_allyears = word_freq_allyears.groupby(['country', 'word']).sum().reset_index()
word_freq_allyears['theme'] = word_freq_allyears['word'].apply(lambda x: get_themes(x))

In [533]:
#df2

In [531]:
india = get_top_percentile_words(word_freq_allyears, "India", 95)
sa = get_top_percentile_words(word_freq_allyears, "South Africa", 80)
usa = get_top_percentile_words(word_freq_allyears, "USA", 94)
uk = get_top_percentile_words(word_freq_allyears, "UK", 94)

df2 = india.append(sa)
df2 = df2.append(usa)
df2 = df2.append(uk)

df2['theme'] = df2['word'].apply(lambda x: get_themes(x))

In [516]:
all_countries = df2.groupby('word').sum().reset_index()
all_countries['country'] = 'All countries'
all_countries['rank'] = all_countries['rank'].rank(method = 'first')
all_countries['theme'] = all_countries['word'].apply(lambda x: get_themes(x))
all_theme = all_countries.copy()
all_countries = all_countries[all_countries['rank']<np.percentile(all_countries['rank'], 50)]

In [520]:
all_theme

,word,count,rank,country,theme
0,abandon,300,1122.0,All countries,No theme
1,abduct,170,754.0,All countries,No theme
2,abortion,889,559.0,All countries,empowerement
3,abuse,2074,79.0,All countries,violence
4,accident,608,1007.0,All countries,No theme
...,...,...,...,...,...
1226,writer,532,1022.0,All countries,No theme
1227,wrong,712,1013.0,All countries,No theme
1228,young,3527,30.0,All countries,No theme
1229,youth,194,759.0,All countries,No theme


In [524]:
all_theme['rank'] = all_theme.groupby('theme').rank(method = 'first', ascending = False)

In [535]:
#all_theme

In [536]:
all_theme[['word', 'theme', 'rank', 'count']].to_csv('../data/processed/words_theme_ranks.csv')

In [544]:
df = all_theme[['word', 'count', 'theme']].pivot(columns = 'word', index = 'theme').reset_index()
df.columns = df.columns.droplevel()
df.rename(columns = {'':'theme'}, inplace = True)
df.to_csv('../data/processed/words_theme_freq.csv')

In [545]:
df2

,country,word,count,theme,rank
20,India,abandon,162,No theme,459.0
27,India,abduct,150,No theme,496.0
52,India,abortion,224,empowerement,303.0
68,India,abuse,630,violence,66.0
84,India,accident,182,No theme,397.0
...,...,...,...,...,...
28937,UK,worker,235,empowerement,243.0
28973,UK,write,340,No theme,146.0
28974,UK,writer,123,No theme,577.0
28976,UK,wrong,275,No theme,194.0


In [514]:
df2.sort_values(by = 'rank')

,country,word,count,theme,rank
35866,USA,man,4144,No theme,1.0
15427,South Africa,man,596,No theme,1.0
23625,UK,man,4697,No theme,1.0
7256,India,man,5559,No theme,1.0
15251,South Africa,kill,521,violence,2.0
...,...,...,...,...,...
32314,USA,documentary,108,No theme,743.0
33727,USA,golf,108,No theme,744.0
34459,USA,identity,108,No theme,745.0
34727,USA,investigation,108,No theme,746.0


In [512]:
word_freq = all_countries.append(df2)[['country', 'word', 'rank']]
word_freq = word_freq.pivot(columns = 'word', index = 'country').reset_index()
word_freq.columns = word_freq.columns.droplevel()
word_freq.rename(columns = {'': 'country'}, inplace = True)
word_freq.to_csv('../data/processed/country_freqtheme_pivoted_all.csv')

In [513]:
df2[['word', 'theme']].drop_duplicates().to_csv('../data/processed/word_themes.csv')

In [377]:
all_countries_theme_rank = all_countries.copy()
all_countries_theme_rank['rank'] = all_countries_theme_rank.groupby('theme').rank(method = 'first')['rank']

In [378]:
all_countries_theme_rank.to_csv('../data/processed/word_theme_ranks.csv')

In [314]:
all_countries_theme_rank.sort_values(by = ['theme', 'rank'])

,word,count,rank,country,theme
682,kill,7713,1.0,All countries,violence
357,die,6744,2.0,All countries,violence
333,death,5919,3.0,All countries,violence
913,police,5903,4.0,All countries,violence
826,murder,4549,5.0,All countries,violence
...,...,...,...,...,...
621,injury,504,72.0,All countries,violence
908,plead,495,73.0,All countries,violence
150,blood,471,74.0,All countries,violence
1024,rob,454,75.0,All countries,violence


In [398]:
df2

,country,word,count,theme,rank
1,India,aadhaar,37,No theme,418.0
9,India,aaradhya,19,No theme,436.0
19,India,aayog,14,No theme,441.0
21,India,abandon,162,No theme,293.0
23,India,abaya,12,No theme,443.0
...,...,...,...,...,...
41720,USA,zolciak,17,No theme,434.0
41721,USA,zombie,22,No theme,429.0
41722,USA,zone,21,No theme,430.0
41723,USA,zoo,96,No theme,355.0


In [4]:
title = headlines[(headlines['headline'].str.startswith('Woman')==True) | (headlines['headline'].str.startswith('Women')==True) ]
title = title[title['year']>=2020].sort_values(by=['country_of_pub', 'monthly_visits'], ascending = False)

In [29]:
title.to_csv('../data/processed/headlines_starting_woman.csv')

In [20]:
women_headlines = title['headline_no_site'].str.replace('Woman', '').str.replace('Women', '')

In [37]:
women_headlines = women_headlines[~(women_headlines.str.startswith("'"))]

In [54]:
women_headlines = women_headlines[~(women_headlines.str.endswith('...'))]
women_headlines = women_headlines[~(women_headlines.str.contains("'"))]

In [56]:
len(list(women_headlines.drop_duplicates()))

27